# Issue Solution: Determining the number of shells

In [41]:
from dmriprep.utils.vectors import DiffusionGradientTable

import numpy as np

In [43]:
dwi = "../../data/sub-02_dwi.nii.gz"
bvec = "../../data/sub-02_dwi.bvec"
bval = "../../data/sub-02_dwi.bval"

In [44]:
np.genfromtxt(bval)

array([   0.   ,    0.   ,    0.   , 3000.   , 3000.   , 2000.   ,
       3000.   , 1000.   , 3000.   , 3000.   , 2000.   , 3000.   ,
       1000.   , 3000.   , 3000.   ,  500.001, 3000.   , 2000.   ,
       3000.   , 3000.   ,  999.999, 3000.   ,  499.999, 3000.   ,
       3000.   , 2000.   , 3000.   , 1000.   , 3000.   , 3000.   ,
       2000.   , 3000.   , 1000.   , 3000.   , 3000.   ,    0.   ,
       3000.   , 2000.   , 3000.   , 1000.   , 3000.   , 3000.   ,
        499.999, 3000.   , 2000.   , 3000.   , 3000.   , 1000.   ,
       3000.   ,    0.   , 3000.   , 3000.   , 2000.   , 3000.   ,
       1000.   , 3000.   , 3000.   , 2000.   , 3000.   , 1000.   ,
       3000.   , 3000.   ,  500.   , 3000.   , 2000.   , 3000.   ,
       3000.   , 1000.   , 3000.   ,    0.   , 3000.   , 2000.   ,
       3000.   , 3000.   ,  999.999, 3000.   ,  500.001, 3000.   ,
       3000.   , 2000.   , 3000.   , 1000.   , 3000.   , 3000.   ,
       2000.   , 3000.   ,  999.999, 3000.   , 3000.   ,    0.

In [45]:
gtab = DiffusionGradientTable(dwi_file=dwi, bvecs=bvec, bvals=bval)

gtab._bvals

array([   0,    0,    0, 3000, 3000, 2000, 3000, 1000, 3000, 3000, 2000,
       3000, 1000, 3000, 3000,  500, 3000, 2000, 3000, 3000, 1000, 3000,
        500, 3000, 3000, 2000, 3000, 1000, 3000, 3000, 2000, 3000, 1000,
       3000, 3000,    0, 3000, 2000, 3000, 1000, 3000, 3000,  500, 3000,
       2000, 3000, 3000, 1000, 3000,    0, 3000, 3000, 2000, 3000, 1000,
       3000, 3000, 2000, 3000, 1000, 3000, 3000,  500, 3000, 2000, 3000,
       3000, 1000, 3000,    0, 3000, 2000, 3000, 3000, 1000, 3000,  500,
       3000, 3000, 2000, 3000, 1000, 3000, 3000, 2000, 3000, 1000, 3000,
       3000,    0, 3000, 2000, 3000, 3000, 1000, 3000,  500, 3000, 3000,
       2000, 3000, 1000, 3000,    0], dtype=uint16)

In [19]:
from dipy.core.gradients import unique_bvals

In [21]:
unique_bvals(gtab._bvals)

array([   0.,  500., 1000., 2000., 3000.])

In [9]:
from collections import Counter

In [10]:
Counter(sorted(gtab._bvals))

Counter({0: 7, 500: 6, 1000: 15, 2000: 15, 3000: 60})

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [2]:
SHELL_DIFF_THRES = 150

In [11]:
def bval_centers(gradient_table, shell_diff_thres=SHELL_DIFF_THRES):
    """
    Parse the available bvals from DiffusionTable into shells

    Parameters
    ----------
    gradient_table : DiffusionGradientTable object
    shell_diff_thres : :obj:`float`
        hfhf
    
    Returns
    -------
    shell_centers : numpy.ndarray of length of bvals vector
        Vector of bvals of shell centers

    Example
    -------

    """
    bvals = diffusion_table.bvals

    # use kmeans to find the shelling scheme
    for k in range(1, len(np.unique(bvals)) + 1):
        kmeans_res = KMeans(n_clusters=k).fit(bvals.reshape(-1, 1))
        if kmeans_res.inertia_ / len(bvals) < shell_diff_thres:
            break
    else:
        raise ValueError(
            'Sorry, bval parsing failed - no shells '
            'are more than {} apart are found'.format(shell_diff_thres)
        )

    # convert the kclust labels to an array
    shells = kmeans_res.cluster_centers_
    shell_centers_vec = np.zeros(bvals.shape)
    for i in range(shells.size):
        shell_centers_vec[kmeans_res.labels_ == i] = shells[i][0]

    return shell_centers_vec

In [33]:
bval_centers(gtab, shell_diff_thres=SHELL_DIFF_THRES)

array([   0.,    0., 3000., 3000., 2000., 3000., 1000., 3000., 3000.,
       2000., 3000., 1000., 3000., 3000.,  500., 3000., 2000., 3000.,
       3000., 1000., 3000.,  500., 3000., 3000., 2000., 3000., 1000.,
       3000., 3000., 2000., 3000., 1000., 3000., 3000.,    0., 3000.,
       2000., 3000., 1000., 3000., 3000.,  500., 3000., 2000., 3000.,
       3000., 1000., 3000.,    0., 3000., 3000., 2000., 3000., 1000.,
       3000., 3000., 2000., 3000., 1000., 3000., 3000.,  500., 3000.,
       2000., 3000., 3000., 1000., 3000.,    0., 3000., 2000., 3000.,
       3000., 1000., 3000.,  500., 3000., 3000., 2000., 3000., 1000.,
       3000., 3000., 2000., 3000., 1000., 3000., 3000.,    0., 3000.,
       2000., 3000., 3000., 1000., 3000.,  500., 3000., 3000., 2000.,
       3000., 1000., 3000.,    0.])

In [33]:
# create mask for each shell